In [ ]:
import os
from rdflib import Graph, Namespace
import ipywidgets as widgets

shacl_output = widgets.Output()

# === SHACL-Regeln anwenden auf hochgeladene Datei ===
def apply_shacl_rules(instance_file_path: str) -> Graph:
    SH = Namespace("http://www.w3.org/ns/shacl#")
    SWEMLS = Namespace("https://w3id.org/semsys/ns/swemls#")
    shapes_dir = "Shapes"  # relative Pfadangabe im Repo

    shape_files = [
        "_generic-shapes.ttl", "A1-shapes.ttl", "A2-shapes.ttl", "A3-shapes.ttl",
        "F1-shapes.ttl", "F2-shapes.ttl", "F3-shapes.ttl", "F4-shapes.ttl",
        "I1-shapes.ttl", "I2-shapes.ttl", "I3-shapes.ttl", "I4-shapes.ttl",
        "I5-shapes.ttl", "I6-shapes.ttl", "I7-shapes.ttl", "O1-shapes.ttl",
        "O2-shapes.ttl", "O3-shapes.ttl", "O4-shapes.ttl", "T1-shapes.ttl",
        "T2-shapes.ttl", "T3-shapes.ttl", "T4-shapes.ttl", "T5-shapes.ttl",
        "T6-shapes.ttl", "T7-shapes.ttl", "T8-shapes.ttl", "T9-shapes.ttl",
        "T10-shapes.ttl", "T11-shapes.ttl", "T12-shapes.ttl", "T13-shapes.ttl",
        "T14-shapes.ttl", "T15-shapes.ttl", "T16-shapes.ttl", "T17-shapes.ttl",
        "T18-shapes.ttl", "T19-shapes.ttl", "T20-shapes.ttl", "T21-shapes.ttl",
        "T22-shapes.ttl", "T23-shapes.ttl", "Y1-shapes.ttl", "Y2-shapes.ttl", "Y4-shapes.ttl"
    ]

    g_instance = Graph()
    g_instance.parse(instance_file_path, format="turtle")

    triples_before = len(g_instance)

    for shape_file in shape_files:
        shape_path = os.path.join(shapes_dir, shape_file)
        g_shape = Graph()

        if not os.path.exists(shape_path):
            with shacl_output:
                print(f"⚠️ Shape file not found: {shape_path}")
            continue

        try:
            g_shape.parse(shape_path, format="turtle")
            with shacl_output:
                print(f"✅ Loaded SHACL shape file: {shape_file}")
        except Exception as e:
            with shacl_output:
                print(f"❌ Error loading {shape_file}: {e}")
            continue

        for rule in g_shape.subjects(predicate=SH.rule, object=None):
            for _, _, construct_query in g_shape.triples((rule, SH.construct, None)):
                query = str(construct_query)
                try:
                    g_instance.update(query)
                except Exception as e:
                    with shacl_output:
                        print(f"❌ Error executing rule from {shape_file}: {e}")

    triples_after = len(g_instance)
    with shacl_output:
        print("✅ All rules applied.")
        print(f"📊 Triples before: {triples_before}")
        print(f"📈 Triples after: {triples_after}")
        print(f"➕ Added: {triples_after - triples_before} triples")

    return g_instance


In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from rdflib import Graph

# === Globale Variablen ===
instance_file_path = None
output = widgets.Output()
result_graph = None

# === Auswahlfeld: Beispiel oder Upload ===
option_selector = widgets.ToggleButtons(
    options=[("Use example file", "example"), ("Upload your own", "upload")],
    description="Select input:",
    style={'description_width': 'initial'}
)

# === Upload-Widget (immer sichtbar) ===
upload_widget = widgets.FileUpload(
    accept='.ttl',
    multiple=False,
    description='Upload TTL file'
)

# === "Continue"-Button ===
continue_button = widgets.Button(description="Continue", button_style='primary')

# === Auswahlhandler ===
def on_option_change(change):
    output.clear_output()
    with output:
        if change['new'] == 'example':
            global instance_file_path
            instance_file_path = "Instance_Files/swemls-instances.ttl"
            print(f"📁 Example file selected:\n→ {instance_file_path}")
        elif change['new'] == 'upload':
            print("📤 Please upload a TTL file using the field below.")

option_selector.observe(on_option_change, names='value')

# === Upload-Handler ===
def on_upload(change):
    global instance_file_path
    if upload_widget.value:
        uploaded = upload_widget.value[0]
        file_name = uploaded['name']
        instance_file_path = file_name
        with open(file_name, 'wb') as f:
            f.write(uploaded['content'])
        output.clear_output()
        with output:
            print(f"✅ File uploaded and saved as:\n→ {file_name}")

upload_widget.observe(on_upload, names='value')

# === Continue-Button-Handler ===
def on_continue(b):
    global result_graph
    output.clear_output()
    if not instance_file_path:
        with output:
            print("⚠️ No file selected or uploaded.")
        return

    if not os.path.exists(instance_file_path):
        with output:
            print(f"❌ File not found on disk:\n→ {instance_file_path}")
        return

    try:
        if instance_file_path.endswith(".ttl") and not instance_file_path.startswith("Instance_Files/"):
            with output:
                print("🔍 SHACL rules will be applied to uploaded file.")
            result_graph = apply_shacl_rules(instance_file_path)
        else:
            g = Graph()
            g.parse(instance_file_path, format="turtle")
            result_graph = g

        with output:
            print(f"✅ RDF file successfully loaded!")
            print(f"📄 Triples in graph: {len(result_graph)}")
            print(f"🔗 Using file:\n→ {instance_file_path}")

    except Exception as e:
        with output:
            print(f"❌ Error parsing TTL file:\n→ {e}")

    # ⬇️ SHACL-Ausgabe sichtbar machen
    display(shacl_output)

continue_button.on_click(on_continue)

# === Anzeige aller Elemente ===
display(option_selector, upload_widget, continue_button, output)


In [ ]:
from rdflib.namespace import RDF
import ipywidgets as widgets
from IPython.display import display

query_interface_output = widgets.Output()
selection_output = widgets.Output()

# Textarea für SPARQL-Query
query_input = widgets.Textarea(
    value="""
PREFIX swemls: <https://w3id.org/semsys/ns/swemls#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?system
WHERE {
  ?system a swemls:System .
  ?system swemls:hasCorrespondingPattern ?pattern .
  FILTER(STRENDS(STR(?pattern), "O1"))
}
""",
    placeholder='Enter your SPARQL query here...',
    description='SPARQL Query:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)

# Button zum Ausführen der Query
run_query_button = widgets.Button(description="Run Query", button_style='primary')

# Dropdown zur Auswahl eines Systems
system_selector = widgets.Dropdown(description="Select system:")
confirm_button = widgets.Button(description="Confirm selection", button_style='success')

# === Globales Ergebnis speichern ===
matched_systems = []

# Handler: Query ausführen
def on_query_run(b):
    global matched_systems
    query_interface_output.clear_output()
    selection_output.clear_output()
    system_selector.options = []

    if 'result_graph' not in globals() or result_graph is None:
        with query_interface_output:
            print("⚠️ RDF graph not loaded.")
        return

    try:
        results = result_graph.query(query_input.value)
    except Exception as e:
        with query_interface_output:
            print(f"❌ Query error: {e}")
        return

    matched_systems = []
    for row in results:
        uri = str(row.system)
        sys_id = uri.split("/")[-1]
        matched_systems.append((sys_id, uri))

    if not matched_systems:
        with query_interface_output:
            print("⚠️ No matching systems found.")
        return

    system_selector.options = [(sys_id, uri) for sys_id, uri in matched_systems]

    with query_interface_output:
        print(f"✅ Found {len(matched_systems)} matching system(s):")
        for i, (sys_id, _) in enumerate(matched_systems):
            print(f" {i+1}: {sys_id}")

    display(system_selector, confirm_button)

# Handler: Auswahl bestätigen
def on_confirm_selection(b):
    global selected_system_id, selected_system_uri
    selected_label = system_selector.label
    selected_uri = system_selector.value
    selected_system_id = selected_label
    selected_system_uri = selected_uri
    with selection_output:
        selection_output.clear_output()
        print(f"✅ You selected: {selected_system_id}")
        print(f"🔗 URI: {selected_system_uri}")

run_query_button.on_click(on_query_run)
confirm_button.on_click(on_confirm_selection)

# === Anzeigen der UI
display(query_input, run_query_button, query_interface_output, selection_output)
